# Bitcoin Key Pairs

In [1]:
# pip install ecdsa

## Manual Key Generation

In [2]:
import ecdsa

# Generate a private key
private_key = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
print("Private key:", private_key.to_string().hex())

# Generate a public key
public_key = private_key.get_verifying_key()
print("Public key: ", public_key.to_string().hex())

Private key: f6f0ec734b569731ac829ede662e2da71e8f771ee02121b4acef981c67200202
Public key:  3cca8e2d954200a3c490fd8adc0daa0188a4cf90774a4cf03b4ccdf016017adcb1d1000065ebed0c23c5052c8b4f34078879bea9b16c0ee113c4b883fccf15e4


While the public key is correct it is not in the proper format. 

## Bitcoin Address

Finding the bitcoin public key or bitcoin address from the private key is a bit more complicated.

The public key is a point on the elliptic curve, but it is not in the proper format. The proper format is a 65 byte array with the first byte being 0x04 and the next 32 bytes being the x coordinate and the last 32 bytes being the y coordinate. The x and y coordinates are 32 bytes each because they are 256 bit numbers. The first byte is 0x04 because it is an uncompressed public key. There are compressed public keys which are 33 bytes long, but we will not be using them.

###. Create a bitcoin address

In [3]:
import hashlib
import base58

# Generate a bitcoin address
# Step 1: Hash public key
sha256 = hashlib.sha256(public_key.to_string()).digest()

# Step 2: Perform RIPEMD-160 hashing on the result of SHA256
ripemd160 = hashlib.new('ripemd160')
ripemd160.update(sha256)
hashed_public_key = ripemd160.digest()

# Step 3: Add version byte in front of RIPEMD-160 hash (0x00 for Main Network)
hashed_public_key = b'\x00' + hashed_public_key

# Step 4: Perform SHA256 hash on the extended RIPEMD-160 result
double_sha256 = hashlib.sha256(hashlib.sha256(hashed_public_key).digest()).digest()

# Step 5: Take the first 4 bytes of the second SHA256 hash. This is the address checksum
address_checksum = double_sha256[:4]

# Step 6: Add the 4 checksum bytes from stage 5 at the end of extended RIPEMD-160 hash from stage 4. This is the 25-byte binary Bitcoin Address.
binary_bitcoin_address = hashed_public_key + address_checksum

# Step 7: Convert the result from a byte string into a base58 string using Base58Check encoding. This is the most commonly used Bitcoin Address format
bitcoin_address = base58.b58encode(binary_bitcoin_address)
print("Bitcoin address: ", bitcoin_address.decode('ascii'))

Bitcoin address:  1LUyLK8jPpPHeXBFK5DWnDKwARVdT6xjXV


In [4]:
# Verify that the bitcoin address is correct
# Step 1: Decode Base58 Bitcoin Address
binary_bitcoin_address_check = base58.b58decode(bitcoin_address)

# Step 2: Extract Bitcoin Address from decoded Base58 Bitcoin Address
binary_bitcoin_address = binary_bitcoin_address_check[:-4]

# Step 3: Extract Bitcoin Address Checksum from decoded Base58 Bitcoin Address
binary_bitcoin_address_checksum = binary_bitcoin_address_check[-4:]

# Step 4: Perform SHA256 hash on the extended RIPEMD-160 result
double_sha256 = hashlib.sha256(hashlib.sha256(binary_bitcoin_address).digest()).digest()

# Step 5: Take the first 4 bytes of the second SHA256 hash. This is the address checksum
address_checksum = double_sha256[:4]

# Step 6: Verify that the address checksum from decoded Base58 Bitcoin Address is equal to the address checksum from the SHA256 hash
print("Bitcoin address is valid:", binary_bitcoin_address_checksum == address_checksum)

Bitcoin address is valid: True


# Libraries for bitcoin keys

### pybitcoin

No longer maintained and does not work with python 3.12

In [5]:
# pip install pybitcoin

In [6]:
# from pybitcoin import BitcoinPrivateKey

# # Generate a private key
# private_key = BitcoinPrivateKey()
# print(f"Private key: {private_key.to_hex()}")

# # Generate a public key
# public_key = private_key.public_key()
# print(f"Public key: {public_key.to_hex()}")

# # Generate a bitcoin address
# bitcoin_address = public_key.address()
# print(f"Bitcoin address: {bitcoin_address}")

### bitcoin-utils

Package appears damaged and does not work with python 3.12

In [7]:
# pip install bitcoin-utils

In [8]:
# from bitcoinutils.keys import PrivateKey, PublicKey

# # Generate a private key
# private_key = PrivateKey()
# print(f"Private key: {private_key.to_wif(compressed=True)}")

# # get the public key
# public_key: PublicKey = private_key.get_public_key()
# print(f"Public key: {public_key.to_hex(compressed=True)}")

# # get the bitcoin address
# bitcoin_address = public_key.get_address()
# print(f"Bitcoin address: {bitcoin_address.to_string()}")
# print(f'Hash160: {bitcoin_address.to_hash160()}')



Sign a message with a private key and verify the signature with the public key.

In [9]:
# message: str = "Hello"
# signature = private_key.sign_message(message)
# print(f"Signature: {signature}")
# # print(f"Signature valid? {public_key.verify(message, signature)}")
# print(f"Signature valid? {public_key.verify_message(public_key.get_address().to_string(), message, signature)}")

### python-bitcoinlib

In [10]:
# pip install python-bitcoinlib

In [11]:
from email import message
import secrets
from bitcoin.wallet import CBitcoinSecret, P2PKHBitcoinAddress
from bitcoin.signmessage import BitcoinMessage, VerifyMessage, SignMessage
from bitcoin.core import x, lx, b2x, b2lx, Hash160

message_: str = "Hello World!"

# Generate a private key
private_key_ = CBitcoinSecret.from_secret_bytes(secrets.token_bytes(32))
print(f"Private key: {private_key_}")

# Generate a public key
public_key_ = private_key_.pub
print(f"Public key: {public_key_.hex()}")

# Generate a bitcoin address
bitcoin_address_ = P2PKHBitcoinAddress.from_pubkey(public_key_.__bytes__())
print(f"Bitcoin address: {bitcoin_address_}")

# Sign message
bitcoin_msg = BitcoinMessage(message_)
signature_ = SignMessage(private_key_, bitcoin_msg)
print(f"Signature: {signature_.decode('utf-8')}")

# Verify message
# create the P2PHK address from the public key
bitcoin_address__ = P2PKHBitcoinAddress.from_pubkey(public_key_.__bytes__())
print(f"Bitcoin address: {bitcoin_address__}")

# verify the message
print(f"Signature valid? {VerifyMessage(bitcoin_address__, bitcoin_msg, signature_)}")

Private key: Kx6zPW7yVcosopZivUNBoqVP8LaxgLynsYieoRVGcgZP78fzKrCh
Public key: 02ec977e80774db675c8cf1110378e4965fc6b91404ea2332789452cd59a48f90e
Bitcoin address: 1H3S8Whw6UvYthk9TqWwupCjX2fU3pZ78D
Signature: H8fo4Dzz8cDIrSzh8n2K8bPt4zzJaVP4it/dkY9ieAXEO2xSyf/cUeQhkkYuu04bDi27ibEDDJ1xf96CpOfKYFo=
Bitcoin address: 1H3S8Whw6UvYthk9TqWwupCjX2fU3pZ78D
Signature valid? True
